In [1]:
import pandas as pd
Phishing_Web = pd.read_csv("C:\\Users\\alfre_g2qn6y7\\OneDrive\\Documents\\dataset_phishing.csv")
Phishing_Web.head()


,url,length_url,length_hostname,ip,nb_dots,nb_hyphens,nb_at,nb_qm,nb_and,nb_or,...,domain_in_title,domain_with_copyright,whois_registered_domain,domain_registration_length,domain_age,web_traffic,dns_record,google_index,page_rank,status
0,http://www.crestonwood.com/router.php,37,19,0,3,0,0,0,0,0,...,0,1,0,45,-1,0,1,1,4,legitimate
1,http://shadetreetechnology.com/V4/validation/a...,77,23,1,1,0,0,0,0,0,...,1,0,0,77,5767,0,0,1,2,phishing
2,https://support-appleld.com.secureupdate.duila...,126,50,1,4,1,0,1,2,0,...,1,0,0,14,4004,5828815,0,1,0,phishing
3,http://rgipt.ac.in,18,11,0,2,0,0,0,0,0,...,1,0,0,62,-1,107721,0,0,3,legitimate
4,http://www.iracing.com/tracks/gateway-motorspo...,55,15,0,2,2,0,0,0,0,...,0,1,0,224,8175,8725,0,0,6,legitimate


In [2]:
X = Phishing_Web.drop(["url","status"],axis=1)
y = Phishing_Web["status"]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
knn = KNeighborsClassifier() 
knn.fit(X_train_scaled, y_train)
y_pred = knn.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9444444444444444


In [5]:
from sklearn.metrics import classification_report
report = classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

  legitimate       0.93      0.96      0.95      1157
    phishing       0.96      0.92      0.94      1129

    accuracy                           0.94      2286
   macro avg       0.95      0.94      0.94      2286
weighted avg       0.95      0.94      0.94      2286



In [6]:
from sklearn.model_selection import KFold, cross_val_score

cv = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(knn, X_train_scaled, y_train, cv=cv, scoring='accuracy')
scores.mean()

0.9339450238259598

The test and cross validation accuracy as well as the f1 score are all in the low to mid 90 percent range. Let's see what happens when we set the distance metric to manhatten by reducing the p value for the minkowski metric from 2 to 1. 

In [107]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
knn = KNeighborsClassifier(p=1, metric = "minkowski") 
knn.fit(X_train_scaled, y_train)
y_pred = knn.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9553805774278216


In [108]:
report = classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

  legitimate       0.95      0.96      0.96      1157
    phishing       0.96      0.95      0.95      1129

    accuracy                           0.96      2286
   macro avg       0.96      0.96      0.96      2286
weighted avg       0.96      0.96      0.96      2286



In [109]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

cv = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(knn, X_train_scaled, y_train_encoded, cv=cv, scoring='accuracy')
scores.mean()

0.9475064395294389

There is some improvement; the test and cross validation accuracy scores as well as the f1 scores are in the mid to high 90 percent range. Now, let's do GridSearch. 

In [124]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
knn = KNeighborsClassifier()
cv = KFold(n_splits=5, shuffle=True, random_state=42)
param_grid = {
    'n_neighbors':np.arange(2,11),  # Number of neighbors (k) 
    'p': [1, 2], # Value for minkowski metric
    'weights': ['uniform', 'distance'] 
}
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=cv, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)


print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")


best_knn = grid_search.best_estimator_


y_pred = best_knn.predict(X_test_scaled)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the best KNN model on the test set: {accuracy:.4f}")

report = classification_report(y_test,y_pred)
print(report)

C:\Users\alfre_g2qn6y7\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:1102: UserWarning: One or more of the test scores are non-finite: [       nan 0.94007008 0.91819782 0.92891471        nan 0.94881875
 0.93689871 0.938867          nan 0.94914704 0.92705577 0.93941351
        nan 0.95133403 0.93394502 0.93842883        nan 0.9510061
 0.92814945 0.93919415        nan 0.95209995 0.93230484 0.93766368
        nan 0.95199084 0.92705583 0.93689811        nan 0.95264706
 0.93011768 0.93580438        nan 0.95199066 0.92530583 0.93580444]
  warnings.warn(


Best parameters: {'n_neighbors': 9, 'p': 1, 'weights': 'distance'}
Best cross-validation score: 0.9526
Accuracy of the best KNN model on the test set: 0.9598
              precision    recall  f1-score   support

  legitimate       0.95      0.97      0.96      1157
    phishing       0.97      0.95      0.96      1129

    accuracy                           0.96      2286
   macro avg       0.96      0.96      0.96      2286
weighted avg       0.96      0.96      0.96      2286



The numbers for test and cross validation accuracy as well as for the f1 score are somewhat better than previous pre-Grid Search results. 

In [20]:
import pandas as pd
Email = pd.read_csv("C:\\Users\\alfre_g2qn6y7\\OneDrive\\Documents\\Phishing_Email.csv")
Email.head()


,Unnamed: 0,Email Text,Email Type,num_chars,num_words,avg_word_len,num_sentences,num_capitals,num_exclamations,num_question_marks,num_special_chars,num_digits,num_urls
0,0,"re : 6 . 1100 , disc : uniformitarianism , re ...",Safe Email,1030.0,230.0,3.482609,9.0,0.0,2.0,0.0,59.0,9.0,0.0
1,1,the other side of * galicismos * * galicismo *...,Safe Email,479.0,91.0,4.274725,6.0,0.0,0.0,2.0,16.0,0.0,0.0
2,2,re : equistar deal tickets are you still avail...,Safe Email,1245.0,305.0,3.085246,7.0,0.0,0.0,1.0,95.0,63.0,0.0
3,3,\nHello I am your hot lil horny toy.\n I am...,Phishing Email,688.0,96.0,5.500000,38.0,39.0,1.0,1.0,110.0,29.0,1.0
4,4,software at incredibly low prices ( 86 % lower...,Phishing Email,441.0,91.0,3.857143,13.0,0.0,0.0,0.0,27.0,2.0,0.0


In [21]:
X = Email.drop(["Unnamed: 0","Email Text","Email Type"],axis=1)
y = Email["Email Type"]
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [22]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
knn = KNeighborsClassifier() 
knn.fit(X_train_scaled, y_train)
y_pred = knn.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.815744920993228


In [23]:
report = classification_report(y_test,y_pred)
print(report)

                precision    recall  f1-score   support

Phishing Email       0.81      0.71      0.76      1429
    Safe Email       0.82      0.89      0.85      2115

      accuracy                           0.82      3544
     macro avg       0.81      0.80      0.80      3544
  weighted avg       0.82      0.82      0.81      3544



In [24]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(knn, X_train_scaled, y_train, cv=cv, scoring='accuracy')
scores.mean()

0.8198555957577366

The results for the test and cross validation accuracy scores are in the low 80 percent range. There is a significant gap between the f1 scores for phishing emails (76%) and safe emails (85%). This is unsurprising given the class imbalance favoring the latter. Now lets do manhatten distance. 

In [25]:
knn = KNeighborsClassifier(p = 1, metric = 'minkowski') 
knn.fit(X_train_scaled, y_train)
y_pred = knn.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.8182844243792325


In [26]:
report = classification_report(y_test,y_pred)
print(report)

                precision    recall  f1-score   support

Phishing Email       0.82      0.70      0.76      1429
    Safe Email       0.82      0.90      0.85      2115

      accuracy                           0.82      3544
     macro avg       0.82      0.80      0.81      3544
  weighted avg       0.82      0.82      0.82      3544



In [27]:
cv = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(knn, X_train_scaled, y_train, cv=cv, scoring='accuracy')
scores.mean()

0.8265593280883801

Some improvement in cross validation and test accuracy score but f1 scores for phishing and safe emails are still identical. Now, let's do Grid Search. 

There is significant improvement for the test and cross validation accuracy scores since they are now approximately 84%. There is also improvement in the f1 score for phishing emails (79%) and safe emails (87%). 

In [130]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
knn = KNeighborsClassifier()
cv = KFold(n_splits=5, shuffle=True, random_state=42)
param_grid = {
    'n_neighbors':np.arange(2,11),  # Number of neighbors (k) 
    'p': [1, 2], # Value for minkowski metric
    'weights': ['uniform', 'distance'] 
}
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=cv, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)


print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")


best_knn = grid_search.best_estimator_


y_pred = best_knn.predict(X_test_scaled)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the best KNN model on the test set: {accuracy:.4f}")

report = classification_report(y_test,y_pred)
print(report)

Best parameters: {'n_neighbors': 10, 'p': 1, 'weights': 'distance'}
Best cross-validation score: 0.8442
Accuracy of the best KNN model on the test set: 0.8437
                precision    recall  f1-score   support

Phishing Email       0.85      0.74      0.79      1429
    Safe Email       0.84      0.91      0.87      2115

      accuracy                           0.84      3544
     macro avg       0.85      0.83      0.83      3544
  weighted avg       0.84      0.84      0.84      3544



In [28]:
import pandas as pd
Webpage_Phishing = pd.read_csv("C:\\Users\\alfre_g2qn6y7\\OneDrive\\Documents\\web-page-phishing.csv")
Webpage_Phishing.head()

,url_length,n_dots,n_hypens,n_underline,n_slash,n_questionmark,n_equal,n_at,n_and,n_exclamation,n_space,n_tilde,n_comma,n_plus,n_asterisk,n_hastag,n_dollar,n_percent,n_redirection,phishing
0,37,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,77,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1
2,126,4,1,2,0,1,3,0,2,0,0,0,0,0,0,0,0,0,1,1
3,18,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,55,2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [29]:
X = Webpage_Phishing.drop("phishing",axis=1)
y = Webpage_Phishing["phishing"]
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,classification_report
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
knn = KNeighborsClassifier() 
knn.fit(X_train_scaled, y_train)
y_pred = knn.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.8854916067146283


In [30]:
report = classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.90      0.92      0.91     12698
           1       0.85      0.83      0.84      7318

    accuracy                           0.89     20016
   macro avg       0.88      0.87      0.88     20016
weighted avg       0.89      0.89      0.89     20016



In [31]:
from sklearn.model_selection import KFold, cross_val_score

cv = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(knn, X_train_scaled, y_train, cv=cv, scoring='accuracy')
scores.mean()

0.885712161579184

The test and cross validation accuracy scores are in the high 80 percent range. The f1 score for non-phishing urls, 0.91, is higher than of phishing urls, 0.84. Let's do Manhatten distance now. 

In [32]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
knn = KNeighborsClassifier(p=1,metric='minkowski') 
knn.fit(X_train_scaled, y_train)
y_pred = knn.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.8861410871302957


In [33]:
report = classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.90      0.92      0.91     12698
           1       0.86      0.83      0.84      7318

    accuracy                           0.89     20016
   macro avg       0.88      0.87      0.88     20016
weighted avg       0.89      0.89      0.89     20016



In [34]:
from sklearn.model_selection import KFold, cross_val_score

cv = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(knn, X_train_scaled, y_train, cv=cv, scoring='accuracy')
scores.mean()

0.8864241237045114

There is a very tiny improvement in cross validation and testing accuracy and no improvement in f1 scores. Let's do GridSearch now. 

In [35]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
knn = KNeighborsClassifier()
cv = KFold(n_splits=5, shuffle=True, random_state=42)
param_grid = {
    'n_neighbors':np.arange(2,11),  # Number of neighbors (k) 
    'p': [1, 2], # Value for minkowski metric
    'weights': ['uniform', 'distance'] 
}
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=cv, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)


print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")


best_knn = grid_search.best_estimator_


y_pred = best_knn.predict(X_test_scaled)


accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the best KNN model on the test set: {accuracy:.4f}")

report = classification_report(y_test,y_pred)
print(report)

Best parameters: {'n_neighbors': 9, 'p': 1, 'weights': 'uniform'}
Best cross-validation score: 0.8887
Accuracy of the best KNN model on the test set: 0.8871
              precision    recall  f1-score   support

           0       0.91      0.92      0.91     12698
           1       0.85      0.83      0.84      7318

    accuracy                           0.89     20016
   macro avg       0.88      0.88      0.88     20016
weighted avg       0.89      0.89      0.89     20016



There is a slight improvement in cross validation and testing accuracy scores compared to the pre-Grid Search results. F1 scores are still identical though. 